In [186]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt


In [187]:
df=pd.read_csv('Data/raw_data.csv')
df.dropna(subset=['Good/Bad'], inplace=True)
df.head()



,Unnamed: 0,Sensor-1,Sensor-2,Sensor-3,Sensor-4,Sensor-5,Sensor-6,Sensor-7,Sensor-8,Sensor-9,...,Sensor-582,Sensor-583,Sensor-584,Sensor-585,Sensor-586,Sensor-587,Sensor-588,Sensor-589,Sensor-590,Good/Bad
0,Wafer-801,2968.33,2476.58,2216.7333,1748.0885,1.1127,100.0,97.5822,0.1242,1.5300,...,NaN,0.5004,0.0120,0.0033,2.4069,0.0545,0.0184,0.0055,33.7876,-1
1,Wafer-802,2961.04,2506.43,2170.0666,1364.5157,1.5447,100.0,96.7700,0.1230,1.3953,...,NaN,0.4994,0.0115,0.0031,2.3020,0.0545,0.0184,0.0055,33.7876,1
2,Wafer-803,3072.03,2500.68,2205.7445,1363.1048,1.0518,100.0,101.8644,0.1220,1.3896,...,NaN,0.4987,0.0118,0.0036,2.3719,0.0545,0.0184,0.0055,33.7876,-1
3,Wafer-804,3021.83,2419.83,2205.7445,1363.1048,1.0518,100.0,101.8644,0.1220,1.4108,...,NaN,0.4934,0.0123,0.0040,2.4923,0.0545,0.0184,0.0055,33.7876,-1
4,Wafer-805,3006.95,2435.34,2189.8111,1084.6502,1.1993,100.0,104.8856,0.1234,1.5094,...,NaN,0.4987,0.0145,0.0041,2.8991,0.0545,0.0184,0.0055,33.7876,-1


In [188]:
def get_cols_with_zero_std_dev(df: pd.DataFrame):
    """
    Returns a list of columns names who are having zero standard deviation.
    """
    cols_to_drop = []
    num_cols = [col for col in df.columns if df[col].dtype != 'O']  # numerical cols only
    for col in num_cols:
        if df[col].std() == 0:
            cols_to_drop.append(col)
    return cols_to_drop

def get_redundant_cols(df: pd.DataFrame, missing_thresh=.7):
    """
    Returns a list of columns having missing values more than certain thresh.
    """
    cols_missing_ratios = df.isna().sum().div(df.shape[0])
    cols_to_drop = list(cols_missing_ratios[cols_missing_ratios > missing_thresh].index)
    return cols_to_drop  

def unnamed_columns(df):
    unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
    return unnamed_columns

In [189]:
cols_to_drop_1 = get_redundant_cols(df,missing_thresh=.7)
cols_to_drop_2 = get_cols_with_zero_std_dev(df)
cols_to_drop_3=  unnamed_columns(df)

drop_cols=cols_to_drop_1+cols_to_drop_2+ cols_to_drop_3

f = open("cols_to_drop.csv", "w")
for each in drop_cols:
    f.write(f"{each},")
f.close()


In [190]:
drop_cols

['Sensor-158',
 'Sensor-159',
 'Sensor-293',
 'Sensor-294',
 'Sensor-6',
 'Sensor-14',
 'Sensor-43',
 'Sensor-50',
 'Sensor-53',
 'Sensor-70',
 'Sensor-75',
 'Sensor-98',
 'Sensor-142',
 'Sensor-150',
 'Sensor-179',
 'Sensor-180',
 'Sensor-187',
 'Sensor-190',
 'Sensor-191',
 'Sensor-192',
 'Sensor-193',
 'Sensor-194',
 'Sensor-195',
 'Sensor-207',
 'Sensor-210',
 'Sensor-227',
 'Sensor-230',
 'Sensor-231',
 'Sensor-232',
 'Sensor-233',
 'Sensor-234',
 'Sensor-235',
 'Sensor-236',
 'Sensor-237',
 'Sensor-238',
 'Sensor-241',
 'Sensor-242',
 'Sensor-243',
 'Sensor-244',
 'Sensor-257',
 'Sensor-258',
 'Sensor-259',
 'Sensor-260',
 'Sensor-261',
 'Sensor-262',
 'Sensor-263',
 'Sensor-264',
 'Sensor-265',
 'Sensor-266',
 'Sensor-267',
 'Sensor-277',
 'Sensor-285',
 'Sensor-314',
 'Sensor-315',
 'Sensor-316',
 'Sensor-323',
 'Sensor-326',
 'Sensor-327',
 'Sensor-328',
 'Sensor-329',
 'Sensor-330',
 'Sensor-331',
 'Sensor-343',
 'Sensor-348',
 'Sensor-365',
 'Sensor-370',
 'Sensor-371',
 'Se

In [191]:
df.isna().sum()

Unnamed: 0    0
Sensor-1      1
Sensor-2      0
Sensor-3      3
Sensor-4      3
             ..
Sensor-587    0
Sensor-588    0
Sensor-589    0
Sensor-590    0
Good/Bad      0
Length: 592, dtype: int64

In [192]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
y

0    -1
1     1
2    -1
3    -1
4    -1
     ..
95   -1
96    1
97   -1
98   -1
99    1
Name: Good/Bad, Length: 100, dtype: int64

In [193]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler, FunctionTransformer

# Define custom function to replace 'NA' with np.nan
def replace_na_with_nan(X):
    return np.where(X == 'NA', np.nan, X)

# Define the steps for the preprocessor pipeline
nan_replacement_step = ('nan_replacement', FunctionTransformer(replace_na_with_nan))
imputer = KNNImputer(n_neighbors=3)
scaler = RobustScaler()
preprocessing_pipeline = Pipeline(
    steps=[('Imputer', imputer), ('Scaler', scaler)])



In [194]:
# import pickle

# with open('preprocessing.pkl', 'wb') as file:
#         pickle.dump(preprocessing_pipeline, file)
        

In [195]:
# with open('preprocessing.pkl', 'rb') as file:
#         model = pickle.load(file)
f = open("cols_to_drop.csv", "r")
drop_cols=f.readlines()
s=str(drop_cols)
cleaned_string = s.strip("[]'")

# Split the cleaned string on commas
split_list = cleaned_string.split(',')

# Remove any leading or trailing spaces from each item in the split list
drop_columns = [item.strip() for item in split_list]
drop_columns = [element for element in drop_columns if element != '']


#X_trans = preprocessing_pipeline.fit_transform(X)
drop_columns



['Sensor-158',
 'Sensor-159',
 'Sensor-293',
 'Sensor-294',
 'Sensor-6',
 'Sensor-14',
 'Sensor-43',
 'Sensor-50',
 'Sensor-53',
 'Sensor-70',
 'Sensor-75',
 'Sensor-98',
 'Sensor-142',
 'Sensor-150',
 'Sensor-179',
 'Sensor-180',
 'Sensor-187',
 'Sensor-190',
 'Sensor-191',
 'Sensor-192',
 'Sensor-193',
 'Sensor-194',
 'Sensor-195',
 'Sensor-207',
 'Sensor-210',
 'Sensor-227',
 'Sensor-230',
 'Sensor-231',
 'Sensor-232',
 'Sensor-233',
 'Sensor-234',
 'Sensor-235',
 'Sensor-236',
 'Sensor-237',
 'Sensor-238',
 'Sensor-241',
 'Sensor-242',
 'Sensor-243',
 'Sensor-244',
 'Sensor-257',
 'Sensor-258',
 'Sensor-259',
 'Sensor-260',
 'Sensor-261',
 'Sensor-262',
 'Sensor-263',
 'Sensor-264',
 'Sensor-265',
 'Sensor-266',
 'Sensor-267',
 'Sensor-277',
 'Sensor-285',
 'Sensor-314',
 'Sensor-315',
 'Sensor-316',
 'Sensor-323',
 'Sensor-326',
 'Sensor-327',
 'Sensor-328',
 'Sensor-329',
 'Sensor-330',
 'Sensor-331',
 'Sensor-343',
 'Sensor-348',
 'Sensor-365',
 'Sensor-370',
 'Sensor-371',
 'Se

In [196]:
X_trans=X.drop(drop_columns,axis=1)
X_trans.head()

,Sensor-1,Sensor-2,Sensor-3,Sensor-4,Sensor-5,Sensor-7,Sensor-8,Sensor-9,Sensor-10,Sensor-11,...,Sensor-581,Sensor-582,Sensor-583,Sensor-584,Sensor-585,Sensor-586,Sensor-587,Sensor-588,Sensor-589,Sensor-590
0,2968.33,2476.58,2216.7333,1748.0885,1.1127,97.5822,0.1242,1.5300,-0.0279,-0.0040,...,NaN,NaN,0.5004,0.0120,0.0033,2.4069,0.0545,0.0184,0.0055,33.7876
1,2961.04,2506.43,2170.0666,1364.5157,1.5447,96.7700,0.1230,1.3953,0.0084,0.0062,...,NaN,NaN,0.4994,0.0115,0.0031,2.3020,0.0545,0.0184,0.0055,33.7876
2,3072.03,2500.68,2205.7445,1363.1048,1.0518,101.8644,0.1220,1.3896,0.0138,0.0000,...,NaN,NaN,0.4987,0.0118,0.0036,2.3719,0.0545,0.0184,0.0055,33.7876
3,3021.83,2419.83,2205.7445,1363.1048,1.0518,101.8644,0.1220,1.4108,-0.0046,-0.0024,...,NaN,NaN,0.4934,0.0123,0.0040,2.4923,0.0545,0.0184,0.0055,33.7876
4,3006.95,2435.34,2189.8111,1084.6502,1.1993,104.8856,0.1234,1.5094,-0.0046,0.0121,...,NaN,NaN,0.4987,0.0145,0.0041,2.8991,0.0545,0.0184,0.0055,33.7876


In [197]:

# with open('preprocessing.pkl', 'rb') as file:
#         pmodel = pickle.load(file)
# scaler=RobustScaler()
# x=scaler.fit_transform(X_trans)
# with open('scaler.pkl', 'wb') as file:
#         pickle.dump(scaler,file)

In [198]:
X_trans=preprocessing_pipeline.fit_transform(X_trans)
X_trans
import pickle

with open('preprocessing.pkl', 'wb') as file:
        pickle.dump(preprocessing_pipeline, file)
 

In [199]:


from imblearn.combine import SMOTETomek

X, y = X_trans, y

resampler = SMOTETomek(sampling_strategy="minority")
X_res, y_res = resampler.fit_resample(X, y)

values_map = {+1: 1, -1: 0}  # Include all possible values in the mapping

# Map the values using the mapping dictionary
y_res = y_res.map(values_map)



In [200]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=1/3, random_state=42)

y_res

0      0
1      1
2      0
3      0
4      0
      ..
183    1
184    1
185    1
186    1
187    1
Name: Good/Bad, Length: 188, dtype: int64

In [201]:
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score,accuracy_score


In [202]:
    
def evaluate_models(X_train,y_train,X_test,y_test, models):
        report = {}
        for i in range(len(list(models))):
            
            model = list(models.values())[i]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            model_score1,model_score2 = roc_auc_score(y_test, y_pred),accuracy_score(y_test,y_pred)

            report[list(models.keys())[i]] = [model_score1,model_score2]

        return report
models = {
                "Random Forest": RandomForestClassifier(),
                "Decision Tree": DecisionTreeClassifier(),
                "Gradient Boosting": GradientBoostingClassifier(),
                "K-Neighbors Classifier": KNeighborsClassifier(),
                "XGBClassifier": XGBClassifier(),
                "AdaBoost Classifier": AdaBoostClassifier(),
                "SVC LINEAR" : SVC(kernel='linear'),
                "SVC RBF" : SVC(kernel='rbf')
            }
model_report: dict = evaluate_models(X_train,y_train,X_test,y_test, models=models)

            ## To get best model score from dict
best_model_score = max(model_report.values())

            ## To get best model name from dict

best_model_name = list(model_report.keys())[
                list(model_report.values()).index(best_model_score)
            ]


In [203]:
print(model_report)

{'Random Forest': [1.0, 1.0], 'Decision Tree': [0.9705882352941176, 0.9682539682539683], 'Gradient Boosting': [0.9705882352941176, 0.9682539682539683], 'K-Neighbors Classifier': [0.7058823529411764, 0.6825396825396826], 'XGBClassifier': [1.0, 1.0], 'AdaBoost Classifier': [1.0, 1.0], 'SVC LINEAR': [0.9705882352941176, 0.9682539682539683], 'SVC RBF': [0.8823529411764706, 0.873015873015873]}


In [204]:
best_model_name

'Random Forest'

In [205]:
best_model_score

[1.0, 1.0]

In [206]:

rf=RandomForestClassifier()

In [207]:
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(roc_auc_score(y_test,y_pred))

1.0
1.0


In [208]:
count_zero = np.count_nonzero(y_pred == 0)
count_one = np.count_nonzero(y_pred == 1)
count_zero,count_one

(34, 29)

In [209]:
with open('fault_detection.pkl', 'wb') as file:
        pickle.dump(rf, file)